Notebook findig water levels in bores that are screened in the Permocarb

Neil Symington
neil.symington@ga.gov.au

In [1]:
%matplotlib inline

In [1]:
from sqlite3 import dbapi2 as sqlite
from shapely.geometry import Polygon, shape
import fiona
import pandas as pd
import numpy as np
from hydrogeol_utils import spatial_functions, AEM_utils, plotting_utils, borehole_utils, SNMR_utils
from geophys_utils._netcdf_point_utils import NetCDFPointUtils
from datetime import datetime
import matplotlib.pyplot as plt
import os, sys
import sqlalchemy as db
from sqlalchemy import create_engine, event
import yaml
import rasterio
from rasterio import Affine
from rasterio.warp import reproject, Resampling
import warnings
import math
warnings.filterwarnings("ignore",category =RuntimeWarning)


C:\Users\u77932\AppData\Local\Continuum\anaconda3\envs\hydrogeol_utils\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [35]:
def point_within_bounds(x,y, bounds):
    if (bounds.left < x) & (bounds.right > x):
        if (bounds.bottom < y) & (bounds.top > y):
            return True
    return False

In [2]:
# Extract data from the database

path = r"\\prod.lan\active\proj\futurex\East_Kimberley\Working\SharedWorkspace\Bores_working\compilation\spatialite"

DB_PATH = os.path.join(path, "East_Kimberley_borehole_data.sqlite")

SPATIALITE_PATH = r'C:\mod_spatialite-4.3.0a-win-amd64'

# Add spatialite dll to path
os.environ['PATH'] = SPATIALITE_PATH + ';' + os.environ['PATH']

engine = db.create_engine('sqlite:///' + DB_PATH, module=sqlite)

@event.listens_for(engine, 'connect')
def connect(dbapi_connection, connection_rec):
    dbapi_connection.enable_load_extension(True)
    dbapi_connection.execute('SELECT load_extension("mod_spatialite")')


connection = engine.connect()


In [24]:
# Custom query so we can join on the standing water level table

query = """

select
   b.borehole_id,
   b.Borehole_name,
   b.Easting,
   b.Northing,
   b.Ground_elevation_mAHD,
   swl.Date,
   swl.Depth,
   constr.*

from

   borehole b
   left join standing_water_level  swl on b.borehole_id = swl.borehole_id
   left join borehole_construction  constr on b.borehole_id = constr.borehole_id
   
where


   b.SWL_available = 1;"""


df_swl = pd.read_sql(query, connection)

# Rename the depth column
df_swl = df_swl.rename(columns = {"Depth": "SWL_depth"})

# Drop columns with no constructino

df_swl = df_swl[df_swl['Construction_name'] == 'screen']

In [25]:
len(df_swl)

49

In [27]:
# Now we want to iterate through the bores and find if the screen if within the 
# Permo-Carboniferous

In [28]:
# Extract the borehole ids into an array

df_swl = df_swl.loc[:,~df_swl.columns.duplicated()]

enos = df_swl['borehole_id'].unique()

In [29]:

# Find the days since the measurement was taken
date_format = "%Y-%m-%d"
today = datetime.now()
days_ago= [(today - datetime.strptime(x, date_format)).days for x in df_swl['Date']]
df_swl['days_since_swl'] = days_ago


In [30]:
# For duplicate entries we will drop the measurements that were taken longer ago

# Minimum indices will be added to a list
min_ind = []

# Iterate through the enos
for item in enos:
    # Get the subset using the eno as a mask
    df_subset = borehole_utils.extract_by_primary_key(df_swl,[item])
    # Find the minumum index using the days since the measurement was takes
    min_ind.append(df_subset['days_since_swl'].argmin())

# Filter usign the minimum index
df_swl = df_swl.loc[min_ind,:] 

C:\Users\u77932\AppData\Local\Continuum\anaconda3\envs\hydrogeol_utils\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.
  # This is added back by InteractiveShellApp.init_path()


In [31]:
# Create a water level elevation column

df_swl['wl_mAHD'] = df_swl['Ground_elevation_mAHD'] - df_swl['SWL_depth']

In [33]:
# OPen gridded datasets

strat_dir = r'\\prod.lan\active\proj\futurex\East_Kimberley\Working\SharedWorkspace\AEM_interp_OrdKeep\strat_grids'

topfile = os.path.join(strat_dir, "Cuk_Pk_top_mAHD.tif")
    
top = rasterio.open(topfile)

top_arr = top.read()[0]

basefile = os.path.join(strat_dir, "Cuk_Pk_base.tif")

base = rasterio.open(basefile)

base_arr = base.read()[0]

top_bounds = 

In [38]:
top.bounds


BoundingBox(left=488209.0001220703, bottom=8268925.0, right=544009.0001220703, top=8343975.0)

In [42]:
# Now we find if the screen is within the Permocarb

df_swl['screened_in_PermoCarb'] = 0

for index, row in df_swl.iterrows():
    x,y = row.Easting, row.Northing
    depth_from, depth_to = row.Depth_from,  row.Depth_to
    
    # Elevations
    elev = row.Ground_elevation_mAHD  
    
    # Elevation from and to
    elev_from, elev_to = elev - depth_from, elev - depth_to
    
    if point_within_bounds(x,y, top.bounds):
        
        top_elev = next(top.sample(np.array([[x,y]])))[0]
        
        base_elev = next(base.sample(np.array([[x,y]])))[0]
        
        if not top_elev < -900:
            
            if not (elev_to > top_elev) or (elev_from < base_elev):
                
                df_swl.at[index, 'screened_in_PermoCarb'] = 1
                
                

    
    
    
    

In [45]:
df_pc_swl = df_swl[df_swl['screened_in_PermoCarb'] == 1]

In [46]:
df_pc_swl.columns

Index(['borehole_id', 'Borehole_name', 'Easting', 'Northing',
       'Ground_elevation_mAHD', 'Date', 'SWL_depth', 'construction_id',
       'Depth_from', 'Depth_to', 'Measurer', 'Construction_name',
       'Construction_type', 'Construction_materials', 'Internal_diameter',
       'Property', 'Property_size', 'days_since_swl', 'wl_mAHD',
       'screened_in_PermoCarb'],
      dtype='object')

In [ ]:
# Plot the bores on a basemap to get an idea of what the swl data looks like

bore_coords = df_pc_swl[['Easting', 'Northing']].values


#swl = df_pc_swl["SWL_depth"].values
wl_mAHD = df_pc_swl["wl_mAHD"].values

plotting_utils.plot_point_dataset(bore_coords, "EPSG:28352", wl_mAHD, 
                   colourbar_label = 'swl', colour_scheme = 'viridis',
                    point_size=100)


In [52]:
df_pc_swl.to_csv(r"\\prod.lan\active\proj\futurex\East_Kimberley\Working\Neil\KeepWaterTable\results\permocarb_bores_trial_2.csv")

In [50]:
# The next approach is using manually picked enos

enos = [626985,
626982,
626989,
626980,
626991,
626987,
626984,
635921,
635732,
635736,
635746,
635735,
635729,
635730,
635738,
635737
]

df_pc_swl = df_swl[df_swl['borehole_id'].isin(enos)]

In [51]:
df_pc_swl

,borehole_id,Borehole_name,Easting,Northing,Ground_elevation_mAHD,Date,SWL_depth,construction_id,Depth_from,Depth_to,Measurer,Construction_name,Construction_type,Construction_materials,Internal_diameter,Property,Property_size,days_since_swl,wl_mAHD,screened_in_PermoCarb
73,635729,RN040879,523301.492,8314853.537,4.575,2018-10-24,2.460,106.0,25.0,31.0,None,screen,inlet,uPVC,0.1000,AP,0.001,212,2.115,1
87,635730,RN040880,523640.009,8316422.357,4.364,2018-11-14,2.215,113.0,16.0,22.0,None,screen,inlet,uPVC,0.1000,AP,0.001,191,2.149,1
106,635732,RN040782,522411.732,8301368.526,12.030,2018-10-29,5.670,45.0,85.8,91.8,None,screen,inlet,uPVC12,0.1010,AP,0.001,207,6.360,0
144,635735,RN040878,505309.448,8293856.701,16.494,2018-09-12,9.948,134.0,36.0,42.0,None,screen,inlet,uPVC,0.1000,AP,0.001,254,6.546,0
162,635736,RN040783,509392.776,8307934.098,16.490,2018-10-30,2.940,85.0,124.0,130.0,None,screen,inlet,uPVC18,0.1143,AP,0.001,206,13.550,1
178,635737,RN040884,507769.229,8299897.749,12.462,2018-11-11,1.250,141.0,29.0,35.0,None,screen,inlet,uPVC,0.1000,AP,0.001,194,11.212,1
192,635738,RN040883,514309.962,8298828.237,17.673,2018-11-11,8.070,148.0,39.0,45.0,None,screen,inlet,uPVC,0.1000,AP,0.001,194,9.603,1
284,635746,RN040870,542730.465,8331303.544,5.055,2018-10-21,-1.140,197.0,85.0,91.0,None,screen,inlet,uPVC,0.1000,AP,0.001,215,6.195,0
372,635921,RN040781,523321.548,8314868.930,4.735,2018-10-28,3.050,34.0,120.0,126.0,None,screen,inlet,uPVC12,0.1010,AP,0.001,208,1.685,1
